In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [2]:
# Read the data
import pandas as pd
df = pd.read_csv('final_dataset.csv')
df.head()

,seeds,lyrics
0,9,"farewell friend, our ties no longer mend this ..."
1,1,we in the building yeah let's go king push the...
2,5,"day that i met you, girl i knew that it was so..."
3,7,living off of what you taught me only what's o...
4,1,there was once in days of yore and in ages and...


In [3]:
# Rename seeds to labels for BERT
df = df.rename(columns={'seeds': 'labels'})

In [6]:
df['lyrics'] = df['lyrics'].astype(str)

In [7]:
# Remove stopwords and stem the words using NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def remove_stopwords_and_stem(text):
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stop_words]
    text = ' '.join(text)
    return text

# Remove numbers and special characters
import re
def remove_numbers_and_special_characters(text):
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', '', text)
    return text

df['lyrics'] = df['lyrics'].apply(remove_numbers_and_special_characters)
df['lyrics'] = df['lyrics'].apply(remove_stopwords_and_stem)

df

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mertbektas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,labels,lyrics
0,9,farewel friend tie longer mend tale come end m...
1,1,build yeah let go king push think know someth ...
2,5,day met girl knew someth special put finger fu...
3,7,live taught top fail see underneath want under...
4,1,day yore age time long gone part cairo merchan...
...,...,...
67,1,vishnuh pervad everywher vashatkaarah invok ob...
68,3,continu journey get eardrum straight pay atten...
69,8,anoth dream blown bit tryin live learn let go ...
70,6,albin lee meldau american aquarium thing chang...


In [ ]:
# Split the data into train, validation and test sets
from sklearn.model_selection import train_test_split

train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df['labels'],
                                                                    random_state=2018,          
                                                                    test_size=0.3,
                                                                    stratify=df['labels'])

val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                                                                random_state=2018,          
                                                                test_size=0.5,
                                                                stratify=temp_labels)

                                                                        